In [2]:
import json
%pip install rouge -q
%pip install sacrebleu -q

# Load the questions
dataset = "../../answer_questions/results/benchmark_results_responses_undergrad.json"

with open(dataset, 'r') as f:
    data = json.load(f)
    
results = data['results']

generated_vs_gold = []
for file in results:
    for question_pos in results[file]:
        generated_vs_gold.append({
            'question': question_pos['question'],
            'gold': question_pos['gold_answer'],
            'generated': question_pos['generated_answer']
        })


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Rouge metric

from rouge import Rouge
import pandas as pd
import sacrebleu

rouge = Rouge()

rouge_scores = []
for q in generated_vs_gold:
    scores = rouge.get_scores(q['generated'], q['gold'])
    rouge_scores.append({
        'question': q['question'],
        'rouge-1': scores[0]['rouge-1'],
        'rouge-2': scores[0]['rouge-2'],
        'rouge-l': scores[0]['rouge-l'],
    })
    
bleu_scores = []
for q in generated_vs_gold:
    bleu = sacrebleu.raw_corpus_bleu([q['generated']], [[q['gold']]])
    bleu_scores.append({
        'question': q['question'],
        'bleu': bleu.score
    })
    
rouge_df = pd.DataFrame(rouge_scores)
bleu_df = pd.DataFrame(bleu_scores)

average_rouge_1 = rouge_df['rouge-1'].apply(lambda x: x['f']).mean()
average_rouge_2 = rouge_df['rouge-2'].apply(lambda x: x['f']).mean()
average_rouge_l = rouge_df['rouge-l'].apply(lambda x: x['f']).mean()

average_bleu = bleu_df['bleu'].mean()

# Display the average scores
print("Average ROUGE-1 F1 Score: {:.4f}".format(average_rouge_1))
print("Average ROUGE-2 F1 Score: {:.4f}".format(average_rouge_2))
print("Average ROUGE-L F1 Score: {:.4f}".format(average_rouge_l))
print("Average BLEU Score: {:.4f}".format(average_bleu))

Average ROUGE-1 F1 Score: 0.4578
Average ROUGE-2 F1 Score: 0.2958
Average ROUGE-L F1 Score: 0.4261
Average BLEU Score: 23.1957


@inproceedings{lin-2004-rouge,
    title = "{ROUGE}: A Package for Automatic Evaluation of Summaries",
    author = "Lin, Chin-Yew",
    booktitle = "Text Summarization Branches Out",
    month = jul,
    year = "2004",
    address = "Barcelona, Spain",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/W04-1013",
    pages = "74--81",
}


@inproceedings{post-2018-call,
  title = "A Call for Clarity in Reporting {BLEU} Scores",
  author = "Post, Matt",
  booktitle = "Proceedings of the Third Conference on Machine Translation: Research Papers",
  month = oct,
  year = "2018",
  address = "Belgium, Brussels",
  publisher = "Association for Computational Linguistics",
  url = "https://www.aclweb.org/anthology/W18-6319",
  pages = "186--191",
}